In [8]:
# 视频帧提取，输出whole picture
import cv2
import os
from tqdm import tqdm

def frame_extractor(input_vid_path, out_dir, order):
    vidcap = cv2.VideoCapture(input_vid_path)
    if vidcap.isOpened():
        pass
    else:
        print("File not found.")
        quit()
    
    # frame_num = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    # 每个视频提取20帧
    req_frames = list(range(0, 80, 4))
    
    for i in range(len(req_frames)):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES,req_frames[i])
        success, frame = vidcap.read()
        if success:
            filename = f"{order}_{i}.png"
            filepath = os.path.join(out_dir, filename)
            cv2.imwrite(filepath, frame)
        else:
            break

    vidcap.release()

files = os.listdir("D:/15341/Desktop/datasets_cite/UADFV/fake_videos/fake")
out_dir = "D:/15341/Desktop/datasets_cite/UADFV/whole picture/fake"
os.makedirs(out_dir, exist_ok=True)
for i, file in enumerate(tqdm(files, desc="total")):
    frame_extractor(f"D:/15341/Desktop/datasets_cite/UADFV/fake_videos/fake/{file}", out_dir, i)

total: 100%|██████████| 49/49 [00:10<00:00,  4.48it/s]


In [3]:
# DeepfakeTIMIT-real
import os
import shutil

out_dir = "D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/whole picture/real"
os.makedirs(out_dir, exist_ok=True)

input_dir = "D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/exact"

order = ["001", "005", "010", "015", "020", "025", "030", "035", "040", "045"]

img_count = 0
for file in os.listdir(input_dir):
    for i in os.listdir(f"{input_dir}/{file}/video"):
        if i == "head" or i == "head2" or i == "head3":
            continue
        for ord, j in enumerate(order):
            shutil.copy(f"{input_dir}/{file}/video/{i}/{j}",f"{out_dir}/{img_count}_{ord}")
        img_count += 1

for file in os.listdir(out_dir):
    os.rename(f"{out_dir}/{file}",f"{out_dir}/{file}.png")

In [4]:
# 提取伪造视频中的帧，输出为图片
import cv2
import os

def frame_extractor(input_dir, out_dir, file, video, order):
    vid = cv2.VideoCapture(f"{input_dir}/{file}/{video}")
    if vid.isOpened():
        pass
    else:
        print("File not found.")
        quit()

    req_frames = list(range(0, 50, 5))

    for i in range(len(req_frames)):
        vid.set(cv2.CAP_PROP_POS_FRAMES,req_frames[i])
        ret,frame = vid.read()
        video = video.split("-")[0]
        cv2.imwrite(f"{out_dir}/{order}_{i}.png",frame)

input_dir = "D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/lower_quality"
out_dir = "D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/whole picture/fake/LQ"
os.makedirs(out_dir, exist_ok=True)

img_count = 0
for file in os.listdir(input_dir):
    if file == ".dircksum":
        continue
    else:
        for i in os.listdir(f"{input_dir}/{file}"):
            if i.split(".")[-1] == "avi":
                frame_extractor(input_dir, out_dir, file, i, img_count)
                img_count += 1

In [1]:
# 提取人脸
import dlib
import cv2
import os
from tqdm import tqdm

def face_extractor(img_path, output_dir, file_name):
    detector = dlib.get_frontal_face_detector()
    image = cv2.imread(img_path)

    # 将图像转换为灰度
    # gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(image)

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        x_hat = int(w / 8)
        y_hat = int(h / 5)
        
        face_image = image[max(0, y-y_hat):min(image.shape[0], (y+h+y_hat)), 
                           max(0, x-x_hat):min(image.shape[1], (x+w+x_hat))]

        resized_image = cv2.resize(face_image, (224, 224))

        filepath = os.path.join(output_dir, file_name)
        cv2.imwrite(filepath, resized_image)

files = os.listdir("D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/whole picture/fake/LQ")
out_dir = "D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/face/fake"
os.makedirs(out_dir, exist_ok=True)
for file in tqdm(files, desc="total"):
    face_extractor(f"D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/whole picture/fake/LQ/{file}", out_dir, file)

# files = os.listdir("D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/whole picture/real")
# out_dir = "D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/face/real"
# os.makedirs(out_dir, exist_ok=True)
# for file in tqdm(files, desc="total"):
#     face_extractor(f"D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/whole picture/real/{file}", out_dir, file)

total: 100%|██████████| 3200/3200 [17:46<00:00,  3.00it/s]


In [9]:
# 处理在截取人脸或眼睛区域时数据缺失的情况
import os

# 加载数据
data_dir = 'D:/15341/Desktop/datasets_cite/DeepfakeTIMIT/face/real'
idx = 1

# for class_name in ['real', 'fake']:
#     label_dir = os.path.join(data_dir, class_name)

for sequence_idx in range(320):
    # if sequence_idx == 33 or sequence_idx == 43:
    #     continue
    
    for image_idx in range(10):
        image_path = os.path.join(data_dir, f'{sequence_idx}_{image_idx}.png')
        if not os.path.exists(image_path):
            print(f"{idx}:file does not exist:{image_path}")
            idx += 1
# 25 88 167 195

In [3]:
"""
In Ictu Oculi: Exposing AI Created Fake Videos by Detecting Eye Blinking
IEEE International Workshop on Information Forensics and Security (WIFS), 2018
Yuezun Li, Ming-ching Chang and Siwei Lyu
"""

import cv2
import numpy as np
import dlib
from tqdm import tqdm
from PIL import Image
import os

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    return coords

def crop_left_eye(img_path, out_dir, file_name):

    filepath = os.path.join(out_dir, file_name)
    if os.path.exists(filepath):
        return 

    front_face_detector = dlib.get_frontal_face_detector()
    lmark_predictor = dlib.shape_predictor('./dlib_model/shape_predictor_68_face_landmarks.dat')

    image = cv2.imread(img_path)
    image = np.uint8(image)
    faces = front_face_detector(image)

    points = None

    if faces is not None or len(faces) > 0:
        for pred in faces:
            points = np.array([[p.x, p.y] for p in lmark_predictor(image, pred).parts()])
            # points = shape_to_np(lmark_predictor(image, pred))
    else:
        raise ValueError('None face or multiple faces')
    
    try:
        left_eye = points[36:42, :]
    except Exception as e:
        return 
    
    lp = np.min(left_eye[:, 0])
    rp = np.max(left_eye[:, 0])
    tp = np.min(left_eye[:, -1])
    bp = np.max(left_eye[:, -1])

    w = rp - lp
    h = bp - tp

    lp_ = int(np.maximum(0, lp - 0.25 * w))
    rp_ = int(np.minimum(image.shape[1], rp + 0.25 * w))
    tp_ = int(np.maximum(0, tp - 1.75 * h))
    bp_ = int(np.minimum(image.shape[0], bp + 1.75 * h))

    eye_img = image[tp_:bp_, lp_:rp_, :]


    size = [128, 128]

    eye_img = cv2.resize(eye_img, tuple(size))
    eye_img = Image.fromarray(eye_img)
    eye_img.save(filepath)

# files = os.listdir("./dataset/UADFV/whole picture/real")
# out_dir = "./dataset/UADFV/eye/real"
# os.makedirs(out_dir, exist_ok=True)
# for file in tqdm(files, desc="total"):
#     crop_left_eye(f"./dataset/UADFV/whole picture/real/{file}", out_dir, file)

files = os.listdir("./dataset/DeepfakeTIMIT/whole picture/real")
out_dir = "./dataset/DeepfakeTIMIT/eye/real"
os.makedirs(out_dir, exist_ok=True)
for file in tqdm(files, desc="total"):
    crop_left_eye(f"./dataset/DeepfakeTIMIT/whole picture/real/{file}", out_dir, file)


total: 100%|██████████| 3200/3200 [44:24<00:00,  1.20it/s]


In [6]:
import dlib
import cv2
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./dlib_model/shape_predictor_68_face_landmarks.dat')

# 读取图像
image = cv2.imread("./dataset/DeepfakeTIMIT/whole picture/real/0_0.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

white_image = np.ones((384, 512, 3), dtype=np.uint8) * 255

# 检测人脸
faces = detector(gray)

# 遍历每张检测到的人脸
for face in faces:
    # 检测关键点
    landmarks = predictor(gray, face)

    # 遍历关键点并在图像上绘制
    for n in range(0, 68):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        cv2.circle(white_image, (x, y), 1, (255, 0, 0), -1)  # 在关键点处绘制小圆点

# 显示或保存输出图像
cv2.imshow("Facial Landmarks", white_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("1.png", white_image)

True